# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- hide: false
- image: images/covid-overview.png
- permalink: /covid-overview/

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create dashboards with fastpages
You see the live version on https://gramener.com/enumter/covid19/ as well.
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create dashboards with fastpages
You see the live version on https://gramener.com/enumter/covid19/ as well.



In [2]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [3]:
#hide

# FETCH
import getpass
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
base_url = '' if (getpass.getuser() == 'Pratap Vardhan') else base_url
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_19-covid-{name}.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

In [4]:
#hide
        
# Confirmed, Recovered, Deaths
df = get_frame('Confirmed')
dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]

LAST_DATE_I = -1
# sometimes last column may be empty, then go backwards
for i in range(-1, -len(dt_cols), -1):
    if not df[dt_cols[i]].fillna(0).eq(0).all():
        LAST_DATE_I = i
        break

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('Deaths')
dft_recovered = get_frame('Recovered')
dfc_cases = dft_cases.groupby('Country/Region')[dt_today].sum()
dfc_deaths = dft_deaths.groupby('Country/Region')[dt_today].sum()
dfc_recovered = dft_recovered.groupby('Country/Region')[dt_today].sum()
dfp_cases = dft_cases.groupby('Country/Region')[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby('Country/Region')[dt_5ago].sum()
dfp_recovered = dft_recovered.groupby('Country/Region')[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, Recovered=dfc_recovered,
    PCases=dfp_cases, PDeaths=dfp_deaths, PRecovered=dfp_recovered))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
for c in 'Cases, Deaths, Recovered'.split(', '):
    df_table[f'{c} (+)'] = df_table[c] - df_table[f'P{c}']
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

,Country/Region,Cases,Deaths,Recovered,PCases,PDeaths,PRecovered,Cases (+),Deaths (+),Recovered (+),Fatality Rate,Continent
0,China,81156,3249,70535,80977,3193,65660,179,56,4875,4.0,Asia
1,Italy,41035,3405,4440,21157,1441,1966,19878,1964,2474,8.3,Europe
2,Iran,18407,1284,5710,12729,611,2959,5678,673,2751,7.0,Asia
3,Spain,17963,830,1107,6391,195,517,11572,635,590,4.6,Europe
4,Germany,15320,44,113,4585,9,46,10735,35,67,0.3,Europe
5,US,13677,200,0,2727,54,12,10950,146,-12,1.5,North America
6,France,10947,243,12,4487,91,12,6460,152,0,2.2,Europe
7,South Korea,8565,91,1540,8086,72,510,479,19,1030,1.1,Asia
8,Switzerland,4075,41,15,1359,13,4,2716,28,11,1.0,Europe
9,United Kingdom,2716,138,67,1144,21,19,1572,117,48,5.1,Europe


In [6]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Recovered', 'Cases (+)', 'Deaths (+)', 'Recovered (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'updated': Timestamp('2020-03-19 00:00:00'),
 'since': Timestamp('2020-03-14 00:00:00'),
 'Cases': 242708,
 'Deaths': 9867,
 'Recovered': 84854,
 'Cases (+)': 86614,
 'Deaths (+)': 4048,
 'Recovered (+)': 12230,
 'China Cases': 81156,
 'China Deaths': 3249,
 'China Recovered': 70535,
 'China Cases (+)': 179,
 'China Deaths (+)': 56,
 'China Recovered (+)': 4875,
 'US Cases': 13677,
 'US Deaths': 200,
 'US Recovered': 0,
 'US Cases (+)': 10950,
 'US Deaths (+)': 146,
 'US Recovered (+)': -12,
 'EU Cases': 108818,
 'EU Deaths': 4875,
 'EU Recovered': 5910,
 'EU Cases (+)': 62323,
 'EU Deaths (+)': 3063,
 'EU Recovered (+)': 3292}

In [7]:
#hide
dft_ct_cases = dft_cases.groupby('Country/Region')[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,1,2,0,0,4,5,5,1,0,0
Albania,0,0,0,0,0,0,0,0,0,0,...,8,2,11,10,5,4,9,4,4,5
Algeria,0,0,0,0,0,0,0,0,0,0,...,0,0,4,2,11,11,6,6,14,13
Andorra,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,37,0,14
Antigua and Barbuda,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [8]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd)
HTML(f'<div>{html}</div>')

Interactive by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)